In [1]:
R1=[0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,4.0,5.0]
R2=[0.6,0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.69,0.7,0.71,0.72,0.73,0.74,0.75,0.76,0.77,0.78,0.79,0.8]

In [2]:
# import common packages
import numpy as np
from qiskit import Aer

# lib from Qiskit Aqua
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA


# lib from Qiskit Aqua Chemistry
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PSI4Driver
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry import MP2Info

In [3]:
    psi4_h2_config = '''
molecule mol {{
   0 1
   H   0.0 0.0 0.0
   H   {} 0.0 0.0
   symmetry c1
}}

set {{
      basis sto-3g
      scf_type pk
      reference uhf
      guess huckel
      guess_mix true
      
}}
'''

In [78]:
molecule=driver.run()

In [4]:
map_type = 'jordan_wigner'
qubit_reduction = False

In [5]:
HF_1=[]
nuclear_repulsion_energy1=[]
Ham1=[]
for i in range(len(R2)):
    length=R2[i]
    driver = PSI4Driver(config=psi4_h2_config.format(length)) 
    molecule=driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    nuclear_repulsion_energy1.append(molecule.nuclear_repulsion_energy)
    HF_1.append(molecule.hf_energy)
    ferOp = FermionicOperator(h1=h1, h2=h2)
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    qubitOp.chop(10**-10)
    Ham1.append(qubitOp)

In [6]:
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

# of electrons: 2
# of spin orbitals: 4


In [7]:
# setup COBYLA optimizer
max_eval = 2000
optimizer = COBYLA(maxiter=max_eval)

# setup HartreeFock state
HF_state = HartreeFock(num_orbitals=num_spin_orbitals, num_particles=num_particles,qubit_mapping=map_type,two_qubit_reduction=False)


# setup UCCSD variational form
uccsd = UCCSD(num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                 initial_state=HF_state, qubit_mapping=map_type,
                 two_qubit_reduction=qubit_reduction, num_time_slices=1)

# backend setting
backend = Aer.get_backend('statevector_simulator')

In [8]:
sto_uccsd=[]
sto_diagext=[]
for i in range(len(Ham1)):
    input_operator=Ham1[i]
    sto_diagext.append(float(NumPyEigensolver(input_operator).run().eigenvalues.real))
    vqe = VQE(operator=input_operator, optimizer=optimizer, quantum_instance=backend)
    vqe.var_form = uccsd
    result = vqe.compute_minimum_eigenvalue()
    sto_uccsd.append(result.eigenvalue.real)
    print('VQE:', result.eigenvalue.real)    

VQE: -1.9982479967770612
VQE: -1.9872171464742334
VQE: -1.9762787658516174
VQE: -1.9654333969247268
VQE: -1.9546815155728465
VQE: -1.944023549850431
VQE: -1.9334599856296277
VQE: -1.9229911873011027
VQE: -1.9126175282580853
VQE: -1.9023393334838286
VQE: -1.8921568759770844
VQE: -1.8820704082346973
VQE: -1.8720800500232317
VQE: -1.8621859535807253
VQE: -1.8523881613713364
VQE: -1.842686661119365
VQE: -1.8330814065820813
VQE: -1.8235722514964046
VQE: -1.8141590112262274
VQE: -1.8048414129728672
VQE: -1.7956191703728566


In [20]:
import pandas as pd
HF=np.array(HF_1)-np.array(nuclear_repulsion_energy1)

dic = {
    "H-H Length": R2, 
    "HF": HF,
    "Ext_DIAG": sto_diagext,
    "UCCSD_VQE": sto_uccsd,
    "Repulsion":nuclear_repulsion_energy1,
    "Energy": np.array(sto_uccsd)+np.array(nuclear_repulsion_energy1) }
df = pd.DataFrame(dic)
df

,H-H Length,HF,Ext_DIAG,UCCSD_VQE,Repulsion,Energy
0,0.60,-1.983090,-1.998248,-1.998248,0.881962,-1.116286
1,0.61,-1.971723,-1.987217,-1.987217,0.867504,-1.119714
2,0.62,-1.960442,-1.976279,-1.976279,0.853512,-1.122767
3,0.63,-1.949247,-1.965433,-1.965433,0.839964,-1.125470
4,0.64,-1.938138,-1.954682,-1.954682,0.826839,-1.127842
5,0.65,-1.927115,-1.944024,-1.944024,0.814119,-1.129905
6,0.66,-1.916180,-1.933460,-1.933460,0.801784,-1.131676
7,0.67,-1.905332,-1.922991,-1.922991,0.789817,-1.133174
8,0.68,-1.894572,-1.912618,-1.912618,0.778202,-1.134416
9,0.69,-1.883900,-1.902339,-1.902339,0.766923,-1.135416


### Select 9 pts and fitting to get $v$ : vibrational frequency

In [35]:
min_index=df[df.Energy==min(df['Energy'])].index.tolist()
min_index=min_index[0]

In [101]:
print(" bond length : ",df['H-H Length'][min_index])

 bond length :  0.73


In [66]:
fitting_E=df['Energy'][min_index-4:min_index+5].tolist()
fitting_length=df['H-H Length'][min_index-4:min_index+5]*1.8897259886
fitting_length.to_list()

In [70]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [74]:
def f_1(x, A, B,C):
    return A * x**2 + B*x +C

In [76]:
z1 = np.polyfit(fitting_length,fitting_E,2) 
p_fit, prov = curve_fit(f_1, fitting_length, fitting_E)

In [86]:
curvature=z1[0]

In [96]:
H_mass=1.00782503223
Li_mass=7.0160034366
reduce_mass=1/(1/H_mass+1/H_mass)

In [97]:
reduce_mass

0.503912516115

In [98]:
Hartree=(curvature*2/reduce_mass/1840)**0.5

In [99]:
v_f=Hartree*219474.63
v_f

5069.711193696306